In [1]:
import pandas as pd
from sklearn.preprocessing import Normalizer, MinMaxScaler, StandardScaler, RobustScaler,QuantileTransformer, PowerTransformer
from sklearn.compose import ColumnTransformer
import keras

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import os

import math


from tensorflow.keras.utils import Sequence
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import datetime as dt

import warnings
warnings.filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  # turn off deprecation warnings



2023-04-16 15:22:12.466917: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-16 15:22:12.531778: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-16 15:22:12.532804: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 15:22:13.464282: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('../data/final_dataset22.csv')
df

,datetime,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,...,NI,RICE,XAU,IRD,SUGAR,ETHANOL,CORN,COTTON,TIN,ZNC
0,1641017460000,47249.44,47281.68,47249.44,47271.88,10.37408,1641017519999,4.903711e+05,403,6.99871,...,0.62547,14.632537,0.000547,4147.926037,0.189924,2.136423,5.928399,1.129187,1.222731,0.102048
1,1641017520000,47271.87,47290.00,47271.87,47289.97,4.11304,1641017579999,1.944688e+05,343,2.50207,...,0.62547,14.632537,0.000547,4147.926037,0.189924,2.136423,5.928399,1.129187,1.222731,0.102048
2,1641017580000,47289.96,47320.00,47287.60,47300.72,13.45235,1641017639999,6.363889e+05,472,6.83725,...,0.62547,14.632537,0.000547,4147.926037,0.189924,2.136423,5.928399,1.129187,1.222731,0.102048
3,1641017640000,47300.71,47324.42,47293.01,47307.04,6.17338,1641017699999,2.920746e+05,416,3.06969,...,0.62547,14.632537,0.000547,4147.926037,0.189924,2.136423,5.928399,1.129187,1.222731,0.102048
4,1641017700000,47307.03,47307.04,47258.73,47259.14,4.41853,1641017759999,2.088964e+05,415,1.66964,...,0.62547,14.632537,0.000547,4147.926037,0.189924,2.136423,5.928399,1.129187,1.222731,0.102048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525525,1672548960000,16521.42,16521.93,16516.49,16518.76,83.64312,1672549019999,1.381688e+06,2545,41.92976,...,29975.00000,18.000000,0.000548,4800.000000,0.200500,2.161000,6.785000,0.830000,19391.166190,3012.500000
525526,1672549020000,16518.76,16519.22,16517.03,16517.41,48.86392,1672549079999,8.071365e+05,1783,21.38648,...,29975.00000,18.000000,0.000548,4800.000000,0.200500,2.161000,6.785000,0.830000,19391.166190,3012.500000
525527,1672549080000,16517.39,16520.64,16517.28,16519.37,61.07865,1672549139999,1.008950e+06,2350,36.47444,...,29975.00000,18.000000,0.000548,4800.000000,0.200500,2.161000,6.785000,0.830000,19391.166190,3012.500000
525528,1672549140000,16519.37,16522.81,16511.92,16521.85,173.39683,1672549199999,2.864058e+06,4050,89.59732,...,29975.00000,18.000000,0.000548,4800.000000,0.200500,2.161000,6.785000,0.830000,19391.166190,3012.500000


In [3]:
df = df.drop(columns = ['ignore','datetime'])
df_copy = df.copy()


In [4]:


# delay: time in future that will be predicted
delay = 3 * 60


# sampling rate: period between timesteps within the sequence
# Sequence with rate=1 : t1,t2...tn
# Sequence with rate=3 : t1,t3...tn*3
sampling_rate = 20

# sequence length: sequence lenght of each sample
half_day = 12 * 60
sequence_length = int(4 * half_day / sampling_rate)


# sequence_stride: period between sequences
# First sequence starts at t0
# Second sequence will start at t1 with sequence_stride=1 or at t5 with sequence_stride=5
sequence_stride = 5



#batch_size: Number of timeseries samples in each batch (except maybe the last one). 
#If None, the data will not be batched (the dataset will yield individual samples).
# Huge impact in performance.
# Tip, should be multiple of 8
batch_size = 32

# Understanding our parameters
msg = f"The timeseries will consist of batches containing {batch_size} sequences of {sequence_length} samples."

msg += f"\nFinally our target is {delay} timesteps in the future"
print(msg)

The timeseries will consist of batches containing 32 sequences of 144 samples.
Finally our target is 180 timesteps in the future


In [5]:
#import matplotlib.pyplot as plt
#loss = history.history["mae"]
#val_loss = history.history["val_mae"]
#epochs = range(1, len(loss) + 1)
#plt.figure()
#plt.plot(epochs, loss, "bo", label="Training MAE")
#plt.plot(epochs, val_loss, "b", label="Validation MAE")
#plt.title("Training and validation MAE")
#plt.legend()
#plt.show()

In [6]:
results = []
results.append(['Scaler', 'MODEL', 'MSE', 'Val MSE', 'MAE', 'Val MAE'])

# Create a list of scalers
scalers = [ 
           ('minmax', MinMaxScaler()), 
           ('robust', RobustScaler()), 
           ('quantile', QuantileTransformer()), 
           ('power', PowerTransformer()), 
           ('normalize', Normalizer())]

# Define a pipeline for each scaler
for scaler_name, scaler in scalers:
    print(scaler_name)

    df = df_copy.copy()

    # Fit the scaler on your dataframe (let's say it's called df)
    df_normalized = scaler.fit_transform(df)

    # Convert the normalized data back to a dataframe
    df_normalized = pd.DataFrame(df_normalized, columns=df.columns)

    # Convert the DataFrame to a NumPy array
    raw_data = df_normalized.values
    close = df_normalized['close'].values
    close = close.reshape((len(close),1))
    # Display the NumPy array
    print(type(raw_data))
    print(raw_data.shape)
    print(close.shape)
    
    print('Prep Data')
    num_train_samples = int(0.6 * len(raw_data))
    num_val_samples = int(0.25 * len(raw_data))
    num_test_samples = len(raw_data) - num_train_samples - num_val_samples
    print("num_train_samples:", num_train_samples)
    print("num_val_samples:", num_val_samples)
    print("num_test_samples:", num_test_samples)

    

    train_dataset = keras.preprocessing.timeseries_dataset_from_array(
                        raw_data[:-delay],
                        targets=close[delay:],
                        sampling_rate=sampling_rate,
                        sequence_stride=sequence_stride,
                        sequence_length=sequence_length,
                        shuffle=False, # Shouldn't the shuffle be set to 0?
                        seed=33,
                        batch_size=batch_size,
                        start_index=0,
                        end_index=num_train_samples)

    print("Done Train")

    val_dataset = keras.preprocessing.timeseries_dataset_from_array(
                        raw_data[:-delay],
                        targets=close[delay:],
                        sampling_rate=sampling_rate,
                        sequence_stride=sequence_stride,
                        sequence_length=sequence_length,
                        shuffle=False,
                        seed=33,
                        batch_size=batch_size,
                        start_index=num_train_samples,
                        end_index=num_train_samples + num_val_samples)

    print("Done Validation")      
        
    test_dataset = keras.preprocessing.timeseries_dataset_from_array(
                        raw_data[:-delay],
                        targets=close[delay:],
                        sampling_rate=sampling_rate,
                        sequence_stride=sequence_stride,
                        sequence_length=sequence_length,
                        shuffle=False,
                        seed=33,
                        batch_size=batch_size,
                        start_index=num_train_samples + num_val_samples)

    print("Done Test")
    
    inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
    
    #print('Training LSTM')####################################################################
    x = layers.LSTM(16)(inputs)
    
    
    outputs = layers.Dense(1)(x)
    model = keras.Model(inputs, outputs)

    callbacks = [
        keras.callbacks.ModelCheckpoint("models/lstm",
       save_best_only=True) 
    ]

    model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

    history = model.fit(train_dataset,
                        epochs=10,
                        validation_data=val_dataset,
                        callbacks=callbacks)
    
    #lstm.compile(optimizer='rmsprop',
                  #loss=['mse', 'mae'],
                  #metrics=['accuracy', 'mse', 'mae'])
    mse = history.history["loss"]
    val_mse = history.history["val_loss"]
    mae = history.history["mae"]
    val_mae = history.history["val_mae"]
    results.append([scaler_name, 'LSTM', mse[-1], val_mse[-1], mae[-1], val_mae[-1]])
    
    
    
    

    

minmax
<class 'numpy.ndarray'>
(525530, 95)
(525530, 1)
Prep Data
num_train_samples: 315318
num_val_samples: 131382
num_test_samples: 78830
Done Train
Done Validation
Done Test


2023-04-16 15:22:24.891929: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-16 15:22:24.893501: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-16 15:22:24.894845: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/10


2023-04-16 15:22:25.182835: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_20' with dtype double and shape [315318,1]
	 [[{{node Placeholder/_20}}]]
2023-04-16 15:22:25.183644: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype int32
	 [[{{node Placeholder/_14}}]]
2023-04-16 15:22:25.548958: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype

1952/1953 [============================>.] - ETA: 0s - loss: 0.0021 - mae: 0.0185

...

1953/1953 [==============================] - 87s 44ms/step - loss: 4.9806e-06 - mae: 0.0022 - val_loss: 4.7058e-06 - val_mae: 0.0022
Epoch 9/10
1952/1953 [============================>.] - ETA: 0s - loss: 4.4563e-06 - mae: 0.0021

2023-04-16 16:43:28.986143: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-16 16:43:28.988170: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-16 16:43:28.989625: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1953/1953 [==============================] - 87s 45ms/step - loss: 4.4562e-06 - mae: 0.0021 - val_loss: 4.2063e-06 - val_mae: 0.0021
Epoch 10/10
1953/1953 [==============================] - ETA: 0s - loss: 4.0090e-06 - mae: 0.0020

2023-04-16 16:44:56.958936: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-16 16:44:56.960725: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-16 16:44:56.962146: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1953/1953 [==============================] - 88s 45ms/step - loss: 4.0090e-06 - mae: 0.0020 - val_loss: 3.8176e-06 - val_mae: 0.0020


In [7]:
print(history.history)

{'loss': [3.923021722584963e-05, 1.716462975309696e-05, 1.1775532584579196e-05, 9.17352554097306e-06, 7.57792622607667e-06, 6.4600558289384935e-06, 5.638401489704847e-06, 4.980646281182999e-06, 4.456194346857956e-06, 4.0090453694574535e-06], 'mae': [0.006101836916059256, 0.004129751119762659, 0.003425187198445201, 0.003023643046617508, 0.0027481026481837034, 0.0025371727533638477, 0.002370251575484872, 0.002227588789537549, 0.0021070234943181276, 0.0019984315149486065], 'val_loss': [2.2331725631374866e-05, 1.3685415069630835e-05, 1.02119920484256e-05, 8.27647545520449e-06, 6.937302714504767e-06, 6.019648026267532e-06, 5.2684654292534105e-06, 4.705841092800256e-06, 4.2062870306835975e-06, 3.817626748059411e-06], 'val_mae': [0.004725642502307892, 0.0036993790417909622, 0.0031956187449395657, 0.002876885002478957, 0.002633875934407115, 0.002453494817018509, 0.00229531223885715, 0.0021692924201488495, 0.002050921320915222, 0.001953874481841922]}


In [12]:
import pandas as pd

df = pd.DataFrame(results)
df

,0,1,2,3,4,5
0,Scaler,MODEL,MSE,Val MSE,MAE,Val MAE
1,minmax,LSTM,0.000238,0.004145,0.009323,0.058902
2,robust,LSTM,0.001643,0.065498,0.0249,0.209585
3,quantile,LSTM,0.000432,0.024432,0.012544,0.135675
4,power,LSTM,0.001399,0.114017,0.024396,0.30615
5,normalize,LSTM,0.000004,0.000004,0.001998,0.001954


In [9]:
print(results)

[['Scaler', 'MODEL', 'MSE', 'Val MSE', 'MAE', 'Val MAE'], ['minmax', 'LSTM', 0.00023810726997908205, 0.004144911654293537, 0.00932299718260765, 0.058901622891426086], ['robust', 'LSTM', 0.0016428339295089245, 0.06549826264381409, 0.024900058284401894, 0.2095847725868225], ['quantile', 'LSTM', 0.00043154225568287075, 0.024431712925434113, 0.012543603777885437, 0.1356746256351471], ['power', 'LSTM', 0.0013993538450449705, 0.11401677876710892, 0.024396225810050964, 0.3061503767967224], ['normalize', 'LSTM', 4.0090453694574535e-06, 3.817626748059411e-06, 0.0019984315149486065, 0.001953874481841922]]


In [ ]:
results = [
        ['Scaler', 'MODEL', 'MSE', 'Val MSE', 'MAE', 'Val MAE']
        , ['minmax', 'LSTM', 0.00023810726997908205, 0.004144911654293537, 0.00932299718260765, 0.058901622891426086]
    , ['robust', 'LSTM', 0.0016428339295089245, 0.06549826264381409, 0.024900058284401894, 0.2095847725868225]
    ,['quantile', 'LSTM', 0.00043154225568287075, 0.024431712925434113, 0.012543603777885437, 0.1356746256351471]
    , ['power', 'LSTM', 0.0013993538450449705, 0.11401677876710892, 0.024396225810050964, 0.3061503767967224]
    , ['normalize', 'LSTM', 4.0090453694574535e-06, 3.817626748059411e-06, 0.0019984315149486065, 0.001953874481841922]]

In [13]:
df.columns = df.iloc[0]

df = df.drop(0)
df

,Scaler,MODEL,MSE,Val MSE,MAE,Val MAE,MSE Index,MAE Index
5,normalize,LSTM,0.000004,0.000004,0.001998,0.001954,0.0,0.00039
1,minmax,LSTM,0.000238,0.004145,0.009323,0.058902,0.000099,0.054914
3,quantile,LSTM,0.000432,0.024432,0.012544,0.135675,0.001054,0.170185
2,robust,LSTM,0.001643,0.065498,0.0249,0.209585,0.01076,0.521867
4,power,LSTM,0.001399,0.114017,0.024396,0.30615,0.015955,0.746891


In [14]:

df.sort_values(by='MSE Index')

,Scaler,MODEL,MSE,Val MSE,MAE,Val MAE,MSE Index,MAE Index
5,normalize,LSTM,0.000004,0.000004,0.001998,0.001954,0.0,0.00039
1,minmax,LSTM,0.000238,0.004145,0.009323,0.058902,0.000099,0.054914
3,quantile,LSTM,0.000432,0.024432,0.012544,0.135675,0.001054,0.170185
2,robust,LSTM,0.001643,0.065498,0.0249,0.209585,0.01076,0.521867
4,power,LSTM,0.001399,0.114017,0.024396,0.30615,0.015955,0.746891
